In [ ]:
# v1
# update file
import re
import json
from google import genai
from google.genai import types

system_prompt = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. Key Term Explanation, explaining terms that are central to Early Buddhist thought and likely unfamiliar to a 10th-grader. won't explain every single
Pali word, only the ones that are crucial for understanding.
When translation, if key terms or concepts that require clarification then add explanations, add explanations will be short and to the point, avoiding philosophical debates.
Example, If the text says "the cessation of craving," I might translate it as: "sự chấm dứt tham ái (craving - sự ham muốn mạnh mẽ)."
But if it says "suffering," I likely won't add an explanation unless the context makes it unclear, because "khổ" is a common word.

4. Glossary (Strict Format): In the end, add The glossary, glossary will only include terms that were actually used and explained within the translated text itself.
**Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`
No generated content: The explanation in Vietnamese must be consistent to the meaning in the text, no unrelated content.

```

<!--pg-->
Từ ngữ:

- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt2 = """Translate the following English text about Early Buddhism into Vietnamese.
The translation should be suitable for a 10th-grade reading level, using simple, modern, and everyday language.

RULE:

- Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

- **Paragraph Structure:** Maintain the original paragraph structure. Do not add or remove any text.

- **Readability:** Use clear, simple Vietnamese. Avoid complex or archaic vocabulary.

- **Key Term Explanations (IN-TEXT):**  Identify terms *crucial* to understanding Early Buddhist thought that a 10th-grader likely wouldn't know. Explain these terms *within the translation itself*, immediately after their first use. Keep explanations short and to the point.
    *   **Format:** `Vietnamese Term (Pali Term - English Term: short definition)`
    *   **Example:** Instead of just "cessation of craving," translate it as:  `sự chấm dứt tham ái (taṇhā - craving: sự ham muốn mãnh liệt, dai dẳng).`
    *   **Do NOT explain very common words** like "suffering" (`khổ`) unless the context is unusual. Only explain terms related to early Buddhism context.

- **Glossary (Strict Format):**
    *   **AFTER** the translation, create a glossary.
    *   **ONLY include terms explained IN-TEXT (Rule 3).**
    *   **Format:** `Vietnamese Term / Pali Term / English Term: Expanded explanation from the in-text definition.`

**Glossary Example (Following a Hypothetical Translation):**

<!--pg-->
Từ ngữ:

- sự chấm dứt tham ái / taṇhā / craving: Sự ham muốn mãnh liệt, dai dẳng, nguyên nhân chính của khổ đau.
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.
- con đường tám nhánh / aṭṭhaṅgika magga / eightfold path: Tám phương diện tu tập để đạt đến giác ngộ.
- giác ngộ / bodhi / enlightenment: Trạng thái hiểu biết hoàn toàn về sự thật.
- thiền định / samādhi / concentration: Tập trung tâm trí vào một đối tượng duy nhất.
"""

system_prompt3 = """Translate content from English to Vietnamese, focusing on Theravada Buddhist texts. Use simple,
understandable, modern Vietnamese terminology for accessibility. For specialized Buddhist terms that are uncommon in
everyday language (such as "attachment" or "complete nirvana" or "purification of doubt"), please provide additional
explanation by including direct English and raw-explaination in vietnam translation in parentheses but only for the first occurrence of each term (explain once).

Example format: "thanh tịnh bằng tri kiến về đạo lộ (purification by knowledge and vision of the way - sự thanh lọc bằng kiến ​​thức và tầm nhìn về đường đạo)"

At the end of each translation, please include a glossary section with the following format:

<!--pg-->
Từ ngữ:
- Vietnamese term / Pali term / English term: Comprehensive explanation in Vietnamese
Example:
- bốn sự thật cao quý / cattāri ariya saccāni / four noble truths: Bốn sự thật mà Đức Phật đã giác ngộ, nền tảng của Phật giáo.

- Always maintain the original meaning while making the translation accessible to modern Vietnamese readers.
When encountering ambiguous terms, provide multiple possible interpretations if necessary.
For quoted passages, preserve the poetic quality where possible while ensuring clarity.
Focus particularly on accuracy with technical dharma terms and use simple, understandable, modern Vietnamese terminology for accessibility.
Maintain the original paragraph structure.
"""
# tempature = 0.5
system_prompt_simple = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.
- For rare or technical Buddhist terms,  translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including:
1. The original English term
2. A classical Vietnamese equivalent (if available)
3. A short, clear explanation (if needed)
Example: *sự vô tâm* (heedlessness - phóng dật); *hướng tâm đúng pháp* (careful attention-như lý tác ý-"Như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ).

- Do not add any introduction or conclusion-translate only.
"""
system_prompt_simple = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.

- For *rare* or *technical Buddhist terms* or *keyword of sentence*,  translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including: (eg: vẻ đẹp)
1. The original English term (eg: feature of beauty)
2. A classical Vietnamese equivalent (if available) (eg: tướng khả ái)
3. A short, clear explanation (if needed)
Example 1: *sự vô tâm* (heedlessness - phóng dật); *hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally-không/như lý tác ý-"như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ).
Example 2: *buồn ngủ và uể oải* (dullness and drowsiness - hôn trầm và thụy miên).

- Do not add any introduction or conclusion-translate only.

-  Pre-defined translation sentence  :
1. I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty -> Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì tăng trưởng.
2. I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it -> Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt

Example:

The Chapter on Giving Up the Hindrances

“Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”
->
Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)
Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm ham muốn dục tham (sensual desire -  kāma-rāga - dục tham : lòng ham muốn vào các đối tượng giác quan như sắc, thanh, hương, vị, xúc,  ý tưởng dễ chịu), chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty - subha nimitta - tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally-không như lý tác ý-"như lý": theo đúng lý, "tác ý": hướng tâm đến, suy nghĩ). đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh.
"""
# tempurature = 0.2
system_prompt_simple2 = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.

- For *rare* or *technical Buddhist terms* or *keyword of sentence*,  translate use clear, modern, and accessible vietnamese words and provide a one-time explanation upon first use, including classic Vietnamese translate: eg1: *vẻ đẹp* (feature of beauty - tịnh tướng); eg2: *hướng tâm đúng/sai pháp* (apply the mind irrationally/rationally-không/như lý tác ý); eg3: *sự buồn ngủ và  tâm trí lờ đờ* (dullness and drowsiness - thīna-middha - hôn trầm và thụy miên)

-  Pre-defined translation sentence  :
1. I do not see a single thing that gives rise to sensual desire, or, when it has arisen, makes it increase and grow like the feature of beauty -> Ta không thấy một pháp nào khác làm *dục tham* chưa sinh sẽ sinh hay đã sinh thì lớn mạnh.
2. I do not see a single thing that prevents ill will from arising, or, when it has arisen, abandons it -> Ta không thấy một pháp nào khác đưa đến *sân hận* chưa sinh sẽ không sinh hay đã sinh thì bị diệt.

At the end of each translation, please include a glossary section with the following format:
<!--pg-->
Từ ngữ:
- Vietnamese term - English term - Pali term : Comprehensive explanation in Vietnamese

Example1 (good, add more information): sân hận - ill will - byāpāda - sân: sự tức giận, căm ghét, ác ý. Sân hận là một trong 5 triều cái (dục, sân, hôn trầm-thùy miên, trạo cử-hối quá, nghi ngờ)
Example2 (ok): sân hận - ill will - byāpāda - sân: sự tức giận, căm ghét, ác ý.
Example3 (bad: not focut): sự bất mãn, lười biếng, ngáp, buồn ngủ sau khi ăn và sự trì trệ của tâm - discontent, sloth, yawning, sleepiness after eating, and mental sluggishness - arati, tandī, jambhā, bhuttasammāda, cetaso līnattam - bất lạc, biếng nhác, ngáp, buồn ngủ sau khi ăn, tâm trì trệ: các biểu hiện của sự uể oải và buồn ngủ.
Example4 (ok): an định của tâm - peace of mind - cetosamatha - tâm an chỉ: trạng thái tâm trí tĩnh lặng, bình yên.

- Do not add any introduction or conclusion-translate only.
- keep all <!--pg--> as is position

Full Example:

The Chapter on Giving Up the Hindrances
<!--pg-->
“1\. Mendicants, I do not see a single thing that gives rise to sensual
desire, or, when it has arisen, makes it increase and grow like the
feature of beauty. When you apply the mind irrationally to the
feature of beauty, sensual desire arises, and once arisen it increases
and grows.”
->>
Chương về Từ Bỏ Các Trở Ngại (Hindrances - Triền Cái)
<!--pg-->
1\. Này các Tỳ-kheo, Ta không thấy một pháp nào khác làm *sự ham muốn* (sensual desire -  kāma-rāga - tham dục) chưa sinh sẽ sinh hay đã sinh thì tăng trưởng và lớn mạnh. Đó là *vẻ đẹp* (feature of beauty - subha nimitta - tướng khả ái). Khi các thầy hướng tâm sai pháp (apply the mind irrationally-không như lý tác ý) đến vẻ đẹp, dục tham chưa sinh sẽ sinh hay đã sinh thì lớn mạnh.

<!--pg-->
Từ ngữ:
- sân hận - ill will - byāpāda - sân: sự tức giận, căm ghét, ác ý. Sân hận là một trong 5 triều cái (dục, sân, hôn trầm-thùy miên, trạo cử-hối quá, nghi ngờ)
- hướng tâm sai pháp - apply the mind irrationally - pali term- không như lý tác ý: "như lý": theo đúng lý, "tác ý": ý nghĩ, hướng tâm đến, suy nghĩ
"""

system_prompt_note = """Translate texts from English to Vietnamese.
The subject matter is Early Buddhism. Translations should be *easy to understand, using simple, modern, everyday language*.

RULE:
1. Readability: Translate the text into Vietnamese that is easy to understand, using simple, modern, everyday language, suitable for a 10th-grade reading level. Avoid complex or archaic vocabulary and sentence structures.

2. Maintain the original paragraph structure; do not add or remove any text.

3. When translation, if key terms or concepts that require clarification, please provide the Vietnamese translation,
followed by the original English word in (), and a brief explanation if necessary.
Example1: `giới cấm thủ (adherence to rules and observances-sự tuân thủ các quy tắc và việc thực hiện các nghi lễ hoặc phong tục)`
Example2: `sân hận (ill will)`
"""

response = None
client = None
def translate_markdown(input_file: str, output_file: str, api_key: str, prompt, model, log, temperature=0):
    """
    Translates an English markdown file to Vietnamese using Google GenAI.

    Args:
        input_file: Path to the input English markdown file (e.g., "abc.md").
        output_file: Path to the output Vietnamese markdown file (e.g., "abc.vn.md").
        api_key: Your Google GenAI API key.
    """

    global response
    global client

    client = genai.Client(api_key=api_key)

    file = client.files.upload(file=input_file)
    # --- Core Translation Logic ---
    try:
        err = ""
        response = client.models.generate_content(
            model= model,
            contents=[file],
            config=types.GenerateContentConfig(
                system_instruction=prompt,
                max_output_tokens=50000,
                # candidates_token_count = 20000,
                # input_token_limit = 20000,
                temperature=temperature
                # temperature=1
                ),
        )

        if response.candidates:
            if response.candidates[0].finish_reason != types.FinishReason.STOP:
                print(f": {input_file} resonse: {response.candidates[0].finish_reason}")
                # log.write(f"{filename}, exception: File not found\n")
                log.write(f": {input_file} resonse {response.candidates[0].finish_reason}")
                #If it is not stop, consider reducing the chunk size
        else: #No candidated return
            pass
            # err = "response.candidates is empty. reduce the chunk size"


        vietnamese_markdown = response.text

    except Exception as e:
      log.write(f'f"Error during translation: {e}')
      print(f'f"Error during translation: {e}')
    finally:
         # --- Output Handling ---
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(vietnamese_markdown)
                log.write(f'{input_file} ok')
        except Exception as e:
            log.write(f'{input_file} Error writing output file: {e}')


    # return f'{input_file} {output_file}:{err}'


import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# peterontech key

_api_key = os.environ.get("GOOGLE_API_KEY")
_temperature = 0.5
_model= "gemini-2.0-pro-exp-02-05"
_model = "gemini-2.5-pro-exp-03-25"
# model ="gemini-2.0-flash-thinking-exp-01-21"
# model = "gemini-2.0-flash"
_model="gemini-2.0-flash"
# _model = "gemini-2.5-flash-preview-04-17"
_prompt = system_prompt3
_prompt = system_prompt_simple2
# "Translate texts from English to Vietnamese. Translations should be *easy to understand, using simple, modern, everyday language*"


<>:129: SyntaxWarning: invalid escape sequence '\.'
<>:129: SyntaxWarning: invalid escape sequence '\.'
/var/folders/0r/x3qsqbx96053svmhzm1mvb_00000gp/T/ipykernel_11257/580313040.py:129: SyntaxWarning: invalid escape sequence '\.'
  system_prompt_simple2 = """Translate English content into Vietnamese, focusing on Theravāda Buddhist texts. Use clear, modern, and accessible Vietnamese.


In [ ]:

def trans_file():

    fname = "045-the-shorter-discourse-on-ways-of-undertaking-things"

    input_filename  = f"../docs/kinhtrungbo/nanamoli-bodhi-en/{fname}.md"
    output_filename = f"../docs/kinhtrungbo/nanamoli-bodhi-vi/{fname}.md"
    # fname = "001"
    input_filename  = "../docs/gameoflife.md"
    output_filename = "../docs/gameoflife.vi.md"


    fi = open("tran_ressult.txt", "a")

    # prompt = system_prompt_note
    translate_markdown(input_filename, output_filename, _api_key, _prompt, _model, fi, _temperature)
    fi.close()

trans_file(?)


In [5]:

# trans folder

def trans_folder(infilelist, output_folder, limit=None, test=False):
    fi = open("tran_ressult.txt", "a")

    count = 1
    for filepath in infilelist: # os.listdir(input_folder):
        filename = os.path.basename(filepath)

        if limit is not None and count >= limit:
            break  # Stop if the limit is reached

        # input_filename = os.path.join(input_folder, filename)
        output_filename = os.path.join(output_folder, filename)

        print(f"Translating {filepath} to {output_filename}")

        try:
            # pass
            if test is False:
                translate_markdown(filepath, output_filename, _api_key, _prompt, _model, fi)
            else:
                print(filepath)
            count += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")  # Handle potential errors
    fi.close()





filelist = [
 '../docs/kinhtangchi/bodhi-en/02-01-entering-upon-the-rains.md',
 '../docs/kinhtangchi/bodhi-en/02-02-disciplinary-issues.md',
 '../docs/kinhtangchi/bodhi-en/02-03-fools.md',
 '../docs/kinhtangchi/bodhi-en/02-04-same-minded.md',
 '../docs/kinhtangchi/bodhi-en/02-05-assemblies.md',
 '../docs/kinhtangchi/bodhi-en/02-06-people.md',
 '../docs/kinhtangchi/bodhi-en/02-07-happiness.md',
 '../docs/kinhtangchi/bodhi-en/02-08-with-a-basis.md',
 '../docs/kinhtangchi/bodhi-en/02-09-dhamma.md',
 '../docs/kinhtangchi/bodhi-en/02-10-fools.md',
 '../docs/kinhtangchi/bodhi-en/02-11-desires.md',
 '../docs/kinhtangchi/bodhi-en/02-12-aspiring.md',
 '../docs/kinhtangchi/bodhi-en/02-13-gifts.md',
 '../docs/kinhtangchi/bodhi-en/02-14-munificence.md',
 '../docs/kinhtangchi/bodhi-en/02-15-meditative-attainment.md',
 '../docs/kinhtangchi/bodhi-en/02-16-anger.md',
 '../docs/kinhtangchi/bodhi-en/02-17-unwholesome-repetition-series.md',
 '../docs/kinhtangchi/bodhi-en/02-18-discipline-repetition-series.md',
 '../docs/kinhtangchi/bodhi-en/02-19-lust-and-so-forth-repetition-serjes.md',
 '../docs/kinhtangchi/bodhi-en/filelist.js',
 '../docs/kinhtangchi/bodhi-en/mucluc.md']

filelist = ['../docs/kinhtangchi/sujato-en/01-001-the-chapter-on-what-occupies-the-mind.md',
 '../docs/kinhtangchi/sujato-en/01-002-the-chapter-on-giving-up-the-hindrances.md',
 '../docs/kinhtangchi/sujato-en/01-003-the-chapter-on-the-useless-mind.md',]

out_folder = '../docs/kinhtangchi/sujato-vi/'
# for filename in sorted([f for f in os.listdir(in_folder) if os.path.isfile(os.path.join(in_folder, f))]):
#     filelist.append(f'{in_folder}{filename}')
# filelist
# filelist = filelist[1:]
trans_folder(infilelist=filelist, output_folder=out_folder)


Translating ../docs/kinhtangchi/sujato-en/01-001-the-chapter-on-what-occupies-the-mind.md to ../docs/kinhtangchi/sujato-vi/01-001-the-chapter-on-what-occupies-the-mind.md
Translating ../docs/kinhtangchi/sujato-en/01-002-the-chapter-on-giving-up-the-hindrances.md to ../docs/kinhtangchi/sujato-vi/01-002-the-chapter-on-giving-up-the-hindrances.md
Translating ../docs/kinhtangchi/sujato-en/01-003-the-chapter-on-the-useless-mind.md to ../docs/kinhtangchi/sujato-vi/01-003-the-chapter-on-the-useless-mind.md


In [3]:
# get all file
import os
filelist = []
# for filename in sorted(os.listdir('../docs/kinhtrungbo/nanamoli-bodhi-en')):
#     filelist.append(filename)

for filename in sorted([f for f in os.listdir('../docs/kinhtangchi/sujato-en') if os.path.isfile(os.path.join('../docs/kinhtangchi/sujato-en', f))]):
    filelist.append('../docs/kinhtangchi/sujato-en/'+filename)

filelist

['../docs/kinhtangchi/sujato-en/01-001-the-chapter-on-what-occupies-the-mind.md',
 '../docs/kinhtangchi/sujato-en/01-002-the-chapter-on-giving-up-the-hindrances.md',
 '../docs/kinhtangchi/sujato-en/01-003-the-chapter-on-the-useless-mind.md',
 '../docs/kinhtangchi/sujato-en/01-004-the-chapter-on-the-wild-mind.md',
 '../docs/kinhtangchi/sujato-en/01-005-the-chapter-on-a-spike.md',
 '../docs/kinhtangchi/sujato-en/01-006-the-chapter-on-a-finger-snap.md',
 '../docs/kinhtangchi/sujato-en/01-007-the-chapter-on-arousing-energy.md',
 '../docs/kinhtangchi/sujato-en/01-008-the-chapter-on-good-friends.md',
 '../docs/kinhtangchi/sujato-en/01-009-the-chapter-on-negligence.md',
 '../docs/kinhtangchi/sujato-en/01-010-the-chapter-on-negligence-2nd.md',
 '../docs/kinhtangchi/sujato-en/01-011-the-chapter-on-not-the-teaching.md',
 '../docs/kinhtangchi/sujato-en/01-012-the-chapter-on-non-offense.md',
 '../docs/kinhtangchi/sujato-en/01-013-the-chapter-on-one-person.md',
 '../docs/kinhtangchi/sujato-en/01-01

In [ ]:
['../docs/kinhtrungbo/nanamoli-bodhi-en/001-the-root-of-all-things.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/002-all-the-taints.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/003-heirs-in-dhamma.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/004-fear-and-dread.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/005-without-blemishes.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/006-if-a-bhikkhu-should-wish.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/007-the-simile-of-the-cloth.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/008-effacement.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/009-right-view.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/010-the-foundations-of-mindfulness.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/011-the-shorter-discourse-on-the-lion-s-roar.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/012-the-greater-discourse-on-the-lion-s-roar.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/013-the-greater-discourse-on-the-mass-of-suffering.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/014-the-shorter-discourse-on-the-mass-of-suffering.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/015-inference.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/016-the-wilderness-in-the-heart.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/017-jungle-thickets.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/018-the-honeyball.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/019-two-kinds-of-thought.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/020-the-removal-of-distracting-thoughts.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/021-the-simile-of-the-saw.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/022-the-simile-of-the-snake.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/023-the-ant-hill.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/024-the-relay-chariots.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/025-the-bait.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/026-the-noble-search.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/027-the-shorter-discourse-on-the-simile-of-the-elephant-s-footprint.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/028-the-greater-discourse-on-the-simile-of-the-elephant-s-footprint.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/029-the-greater-discourse-on-the-simile-of-the-heartwood.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/030-the-shorter-discourse-on-the-simile-of-the-heartwood.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/031-the-shorter-discourse-in-gosinga.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/032-the-greater-discourse-in-gosinga.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/033-the-greater-discourse-on-the-cowherd.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/034-the-shorter-discourse-on-the-cowherd.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/035-the-shorter-discourse-to-saccaka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/036-the-greater-discourse-to-saccaka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/037-the-shorter-discourse-on-the-destruction-of-craving.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/038-the-greater-discourse-on-the-destruction-of-craving.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/039-the-greater-discourse-at-assapura.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/040-the-shorter-discourse-at-assapura.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/041-the-brahmins-of-sala.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/042-the-brahmins-of-veranja.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/043-the-greater-series-of-questions-and-answers.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/044-the-shorter-series-of-questions-and-answers.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/045-the-shorter-discourse-on-ways-of-undertaking-things.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/046-the-greater-discourse-on-ways-of-undertaking-things.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/047-the-inquirer.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/048-the-kosambians.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/049-the-invitation-of-a-brahma.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/050-the-rebuke-to-mara.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/051-to-kandaraka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/052-the-man-from-atthakanagara.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/053-the-disciple-in-higher-training.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/054-to-potaliya.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/055-to-jivaka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/056-to-upali.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/057-the-dog-duty-ascetic.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/058-to-prince-abhaya.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/059-the-many-kinds-of-feeling.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/060-the-incontrovertible-teaching.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/061-advice-to-rahula-at-ambalatthika.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/062-the-greater-discourse-of-advice-to-rahula.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/063-the-shorter-discourse-to-malunkyaputta.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/064-the-greater-discourse-to-malunkyaputta.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/065-to-bhaddali.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/066-the-simile-of-the-quail.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/067-at-catuma.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/068-at-nalakapana.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/069-gulissani.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/070-at-kitagiri.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/071-to-vacchagotta-on-the-threefold-true-knowledge.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/072-to-vacchagotta-on-fire.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/073-the-greater-discourse-to-vacchagotta.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/074-to-dighanakha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/075-to-magandiya.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/076-to-sandaka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/077-the-greater-discourse-to-sakuludayin.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/078-samanamanikaputta.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/079-the-shorter-discourse-to-sakuludayin.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/080-to-vekhanassa.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/081-ghatikara-the-potter.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/082-on-ratthapala.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/083-king-makhadeva.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/084-at-madhura.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/085-to-prince-bodhi.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/086-on-angulimala.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/087-born-from-those-who-are-dear.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/088-the-cloak.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/089-monuments-to-the-dhamma.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/090-at-kannakatthala.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/091-brahmayu.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/092-to-sela.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/093-to-assalayana.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/094-to-ghotamukha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/095-with-canki.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/096-to-esukari.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/097-to-dhananjani.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/098-to-vasettha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/099-to-subha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/100-to-sangarava.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/101-at-devadaha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/102-the-five-and-three.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/103-what-do-you-think-about-me.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/104-at-samagama.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/105-to-sunakkhatta.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/106-the-way-to-the-imperturbable.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/107-to-ganaka-moggallana.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/108-with-gopaka-moggallana.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/109-the-greater-discourse-on-the-full-moon-night.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/110-the-shorter-discourse-on-the-full-moon-night.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/111-one-by-one-as-they-occurred.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/112-the-sixfold-purity.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/113-the-true-man.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/114-to-be-cultivated-and-not-to-be-cultivated.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/115-the-many-kinds-of-elements.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/116-isigili-the-gullet-of-the-seers.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/117-the-great-forty.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/118-mindfulness-of-breathing.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/119-mindfulness-of-the-body.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/120-reappearance-by-aspiration.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/121-the-shorter-discourse-on-voidness.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/122-the-greater-discourse-on-voidness.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/123-wonderful-and-marvellous.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/124-bakkula.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/125-the-grade-of-the-tamed.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/126-bhumija.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/127-anuruddha.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/128-imperfections.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/129-fools-and-wise-men.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/130-the-divine-messengers.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/131-one-fortunate-attachment.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/132-ananda-and-one-fortunate-attachment.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/133-maha-kaccana-and-one-fortunate-attachment.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/134-lomasakangiya-and-one-fortunate-attachment.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/135-the-shorter-exposition-of-action.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/136-the-greater-exposition-of-action.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/137-the-exposition-of-the-sixfold-base.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/138-the-exposition-of-a-summary.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/139-the-exposition-of-non-conflict.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/140-the-exposition-of-the-elements.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/141-the-exposition-of-the-truths.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/142-the-exposition-of-offerings.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/143-advice-to-anathapinika.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/144-advice-to-channa.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/145-advice-to-punna.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/146-advice-from-nandaka.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/147-the-shorter-discourse-of-advice-to-rahula.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/148-the-six-sets-of-six.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/149-the-great-sixfold-base.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/150-to-the-nagaravindans.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/151-the-purification-of-almsfood.md',
 '../docs/kinhtrungbo/nanamoli-bodhi-en/152-the-development-of-the-faculties.md',
]

In [ ]:
print(1+1000)